In [ ]:
import numpy as np
import pandas as pd
import gc
import math
import os.path
import time
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
from dateutil import parser
from sklearn.preprocessing import StandardScaler, RobustScaler
from tqdm import tqdm
import copy
from statsmodels.tsa.arima_model import ARIMA
import warnings
warnings.filterwarnings("ignore")

##풀매수 or 대기 #수수료 때문에 거래가 많으면 손해
##항상 119에 매도 #119가 제일 높기때문 #상승확률에 따라 바꿔주는게 좋으나 설계가 어려움
# 24시간으로 할지, 6시간정도로 나눌것인지 #데이터가 너무 부족
# gan이나 리샘플링으로 데이터 늘리기

#코랩에서 작성 및 실행. 파일은 구글드라이브에서 불러오기
train_x = pd.read_csv("drive/MyDrive/train_x_df.csv") 
train_y = pd.read_csv("drive/MyDrive/train_y_df.csv")
test_x = pd.read_csv("drive/MyDrive/test_x_df.csv")

In [ ]:
#1. 컬럼정리

def heikin(df):
  df['Ho'] = (df['open'].shift(1) - df['close'].shift(1)) / 2
  df['Hc'] = (df['open'] + df['close'] + df['high'] + df['low']) / 4
  df['Hc-Ho'] = (df['Hc'] - df['Ho'])
#  df['h-l'] = df['high'] - df['low']
  df['h-o'] = df['high'] - df['Hc']
  df['l-o'] = df['low'] - df['Hc']
  df['v/t'] = df['volume'] / df['trades']
  df['v/q'] = df['volume'] / df['quote_av']
  df['tbb/v'] = df['tb_base_av'] / df['volume']
  df['v'] = df['volume']
  df['t'] = df['trades']
  del df['open']
  del df['close']
  del df['high']
  del df['low']
  del df['quote_av']
  del df['tb_base_av']
  del df['tb_quote_av']
  del df['volume']
  del df['trades']
#df.drop(['high','low','close','quote_av','tb_base_av','tb_quote_av'],axis=1)

heikin(train_x)
heikin(test_x)
print(train_x.columns)

train_x = train_x[train_x.time!=0]
test_x = test_x[test_x.time!=0]

train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [ ]:
#2. 스케일

r = RobustScaler()

train_without_target = train_x.drop(['sample_id', 'time', 'coin_index'],axis=1)
train_scaled = r.fit_transform(train_x[['Ho', 'Hc', 'Hc-Ho', 'h-o', 'l-o',
       'v/t', 'v/q', 'tbb/v', 'v', 't']])
train_x = train_x.drop(['Ho', 'Hc', 'Hc-Ho', 'h-o', 'l-o',
       'v/t', 'v/q', 'tbb/v', 'v', 't'],axis=1)
train_scaled = pd.DataFrame(train_scaled, columns=train_without_target.columns, index=list(train_x.index.values))
train_x = pd.concat([train_x, train_scaled], axis=1)

train_without_target = test_x.drop(['sample_id', 'time', 'coin_index'],axis=1)
train_scaled = r.fit_transform(test_x[['Ho', 'Hc', 'Hc-Ho', 'h-o', 'l-o',
       'v/t', 'v/q', 'tbb/v', 'v', 't']])
test_x = test_x.drop(['Ho', 'Hc', 'Hc-Ho', 'h-o', 'l-o',
       'v/t', 'v/q', 'tbb/v', 'v', 't'],axis=1)
train_scaled = pd.DataFrame(train_scaled, columns=train_without_target.columns, index=list(test_x.index.values))
test_x = pd.concat([test_x, train_scaled], axis=1)

In [ ]:
# 3. 2차원을 3차원으로

def stf(df): #피쳐사이즈/타임사이즈/샘플사이즈
  f_size = df.iloc[:,2:].shape[1] 
  t_size = len(df.time.value_counts())
  s_size = len(df.sample_id.value_counts())
  return df.iloc[:,2:].values.reshape([s_size, t_size, f_size])

train_xa, train_ya, test_xa = stf(train_x), stf(train_y), stf(test_x)

###
def plot_series(x_series, y_series):
    plt.plot(x_series, label = 'input_series')
    plt.plot(np.arange(len(x_series), len(x_series)+len(y_series)),
             y_series, label = 'output_series')
    plt.axhline(1, c = 'red')
    plt.legend()
idx = 1
plot_series(train_xa[idx,:,3], train_ya[idx,:,3])
plt.show()

In [ ]:
#4. 예측
sp = 3000
val_x = train_x[train_x.sample_id < sp]
val_y = train_y[train_y.sample_id < sp]

val_xa = stf(val_x)
val_ya = stf(val_y)
valid_pred_array = np.zeros([sp, 120])

for idx in tqdm(range(val_xa.shape[0])):
    try:
        try:
            x_series = val_xa[idx,:,1]

            model = ARIMA(x_series, order=(5,1,1))
            fit  = model.fit()
            preds = fit.predict(1,120, typ='levels')
            valid_pred_array[idx,:] = preds # - (preds[0]-x_series[-1])
        except:
            print("order 4 1 1")
            x_series = val_xa[idx,:,1]

            model = ARIMA(x_series, order=(4,1,1))
            fit  = model.fit()
            preds = fit.predict(1,120, typ='levels')
            valid_pred_array[idx,:] = preds
    except:
        print(idx, " 샘플은 수렴하지 않습니다.")
        # ARIMA의 (p,d,q) 값이 (5,1,1), (4,1,1)에서 수렴하지 않을 경우
        # 모두 0으로 채움
        pass

In [ ]:
#5. 예측값 중 제일 고가의 시간에 매도

def array_to_submission(x_array, pred_array):
    # 입력 x_arrry와 출력 pred_arry를 통해서 
    # buy_quantitiy와 sell_time을 결정
    submission = pd.DataFrame(np.zeros([pred_array.shape[0],2], np.int64),
                columns = ['buy_quantity', 'sell_time'])
    submission = submission.reset_index()
    submission.loc[:, 'buy_quantity'] = 0.1
    
    buy_price = []
    for idx, sell_time in enumerate(np.argmax(pred_array, axis = 1)):
        buy_price.append(pred_array[idx, sell_time])
    buy_price = np.array(buy_price)
    # 115% 이상 상승한하고 예측한 sample에 대해서만 100% 매수
    submission.loc[:, 'buy_quantity'] = (buy_price > 1.1) * 1
    # 모델이 예측값 중 최대 값에 해당하는 시간에 매도
    submission['sell_time'] = np.argmax(pred_array, axis = 1)
    submission.columns = ['sample_id','buy_quantity', 'sell_time']
    return submission

vsub = array_to_submission(val_xa, valid_pred_array)
vsub.buy_quantity.value_counts()

In [ ]:
def df2d_to_answer(df_2d):
    # valid_y_df로부터
    # open 가격 정보가 포함된
    # [샘플 수, 120분] 크기의 
    # 2차원 array를 반환하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    sample_index = df_2d.sample_id.value_counts().index
    array_2d = df_2d.open.values.reshape([sample_size, time_size])
    sample_index = list(sample_index)
    return array_2d, sample_index

def COIN(y_df, submission, df2d_to_answer = df2d_to_answer):
    # 2차원 데이터프레임에서 open 시점 데이터만 추출하여 array로 복원
    # sample_id정보를 index에 저장
    y_array, index = df2d_to_answer(y_df)
    
    # index 기준으로 submission을 다시 선택
    submission = submission.set_index(submission.columns[0])
    submission = submission.iloc[index, :]    
    
    # 초기 투자 비용은 10000 달러
    total_momey      = 10000 # dolors
    total_momey_list = []
    
    # 가장 처음 sample_id값
    start_index = submission.index[0]
    for row_idx in submission.index:
        sell_time  = submission.loc[row_idx, 'sell_time']
        buy_price  = y_array[row_idx - start_index, 0]
        sell_price = y_array[row_idx - start_index, sell_time]
        buy_quantity = submission.loc[row_idx, 'buy_quantity'] * total_momey
        residual = total_momey - buy_quantity
        ratio = sell_price / buy_price
        total_momey = buy_quantity * ratio * 0.9995 * 0.9995 + residual        
        total_momey_list.append(total_momey)
        
    return total_momey, total_momey_list

total_momey, total_momey_list = COIN(val_y, vsub)
print(total_momey)

plt.plot(total_momey_list)
plt.title("history")
plt.show()

In [ ]:
#7. 테스트 예측 및 기록

test_pred_array = np.zeros([test_xa.shape[0],
                           120])
for idx in tqdm(range(test_xa.shape[0])):
    try:
        try:
            x_series = test_xa[idx,:,1]

            model = ARIMA(x_series, order=(5,1,1))
            fit  = model.fit()
            preds = fit.predict(1,120, typ='levels')
            test_pred_array[idx,:] = preds
        except:
            x_series = test_xa[idx,:,1]
            model = ARIMA(x_series, order=(4,1,1))
            fit  = model.fit()
            preds = fit.predict(1,120, typ='levels')
            test_pred_array[idx,:] = preds
    except:
        print(idx, " 샘플은 수렴하지 않습니다.")
        pass